# Pràctica 2: Recomanador Simple

Nom dels alumnes del grup:


## 1. INTRODUCCIÓ

### 1.1. Abans de començar...

**\+ A més a més de les que ja es troben presents en la 1a cel·la i funcions natives de Python, durant la pràctica, només es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades.

### 1.2. Dades: puntuacions de pel·licules

La base de dades [movielens-1M](http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·lícules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online [MovieLens](http://www.movielens.org/).

El consum total de tots els usuaris s'hi pot trobar al document "ratings.dat" el format següent:

    UserID::MovieID::Rating::Timestamp

- **UserID** usuari, amb id's entre 1 i 6040
- **MovieID** pel·licula, amb id's entre 1 i 3952
- **Rating** puntuació, en una escala de 1 a 5 estrelles.
- **Timestamp** representat en segons

Cada usuari té com a mínim 20 puntuacions.

### 1.3. Dades: usuaris



Al fitxer ``users.dat`` hi trobem la informació referent a cadascun dels usuaris en el següent format:

        UserID::Gender::Age::Occupation::Zip-code

- **Gender** ve donat per "M" per home i "F" per dona.
- **Age** està representada de la següent forma:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- **Occupation** es tria entre les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

Els usuaris han donat la informació voluntariament. Així doncs, alguns usuaris poden no tenir informació.


### 1.4. Dades: pel·lícules



Al fitxer ``movies.dat`` hi trobem la informació referent a cadascuna de les películes en el següent format:

        MovieID::Title::Genres

- **Titles** són identics als titols de la base de dades IMDB, incloent l'any de llançament.
- **Genres** de les películes estan separats i seleccionats d'entre els següents:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

Algunes películes poden tenir l'ID malament degut a duplicats accidentals.

Les películes s'han entrat manualment, així que poden existir altres inconsistencies.

## 2. Exploració de les dades

### 2.1. Descarregar i llegir dades

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball.

In [1]:
import os
if os.path.isfile("/etc/password.txt") == False:
    os.system('wget -nc http://files.grouplens.org/datasets/movielens/ml-1m.zip')
    os.system('unzip ml-1m.zip')

--2023-11-24 17:25:04--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 

128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5,6M) [application/zip]
Saving to: ‘ml-1m.zip’

     0K .......... .......... .......... .......... ..........  0%  122K 47s
    50K .......... .......... .......... .......... ..........  1%  244K 35s
   100K .......... .......... .......... .......... ..........  2%  222M 23s
   150K .......... .......... .......... .......... ..........  3% 51,6M 17s
   200K .......... .......... .......... .......... ..........  4%  245K 18s
   250K .......... .......... .......... .......... ..........  5%  156M 15s
   300K .......... .......... .......... .......... ..........  6% 96,3M 13s
   350K .......... .......... .......... .......... ..........  6%  122K 17s
   400K .......... .......... .......... .......... ..........  7%  162M 15s
   450K .......... .......... .......... .......... ..........  8%  193M 13s
   500K ......

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


.... .......... .......... .......... 98%  245K 0s
  5700K .......... .......... .......... .......... .......... 99%  193M 0s
  5750K .......... .......... ........                        100% 18,8M=7,3s

2023-11-24 17:25:13 (795 KB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [2]:
import math
import numpy as np
import pandas as pd
import datetime
import itertools
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

In [3]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')


### 2.2. Inspecció de les taules

In [4]:
users[:10] 

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [5]:
users[-10:]

,user_id,gender,age,occupation,zip
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [6]:
ratings[-10:]

,user_id,movie_id,rating,timestamp
1000199,6040,2022,5,956716207
1000200,6040,2028,5,956704519
1000201,6040,1080,4,957717322
1000202,6040,1089,4,956704996
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [7]:
ratings[:10]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [8]:
ratings.sort_values('movie_id')[:5]


,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [9]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 2.3 **Exemple:** Com extreure informació d'un DataFrame.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**, dades que estan a frames diferents.

El primer pas a obtenir una única estructura que contingui tota la informació. Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció.

Reviseu aquests conceptes de pandas: https://pandas.pydata.org/docs/user_guide/merging.html

In [11]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificador d'usuari
data.sort_values(by='user_id')[:10]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
28501,1,48,5,978824351,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance
13819,1,938,4,978301752,F,1,10,48067,Gigi (1958),Musical
51327,1,1207,4,978300719,F,1,10,48067,To Kill a Mockingbird (1962),Drama
31152,1,1721,4,978300055,F,1,10,48067,Titanic (1997),Drama|Romance
37916,1,2762,4,978302091,F,1,10,48067,"Sixth Sense, The (1999)",Thriller
18472,1,2687,3,978824268,F,1,10,48067,Tarzan (1999),Animation|Children's
45685,1,2692,4,978301570,F,1,10,48067,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance
22832,1,720,3,978300760,F,1,10,48067,Wallace & Gromit: The Best of Aardman Animatio...,Animation
32771,1,745,3,978824268,F,1,10,48067,"Close Shave, A (1995)",Animation|Comedy|Thriller


La funció ``iloc`` ens permet obtenir un subconjunt de files i/o columnes indexades per un enter:

In [12]:
data.iloc[3:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


Els índexs Booleans ens permeten seleccionar una part de la taula que compleix una condició.

In [13]:
# comptem quin tant per cent de ratings estan fets per una dona

print(data[data['gender']=='F']['rating'].count()/float(data['rating'].count())*100, '%')

24.638850480249626 %


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes.

Reviseu aquests conceptes:
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table

In [14]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

age,1,18,25,35,45,50,56
title,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.555556,4.333333
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.500000,2.666667
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.208333,2.666667
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.551724,3.928571
1-900 (1994),NaN,NaN,2.000000,NaN,NaN,NaN,3.000000
10 Things I Hate About You (1999),3.745455,3.415020,3.432950,3.102941,3.258065,3.629630,4.000000
101 Dalmatians (1961),3.514286,3.295082,3.613757,3.826087,3.976744,3.650000,3.190476
101 Dalmatians (1996),3.088235,2.467742,2.928571,3.279570,3.482759,3.400000,3.555556


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [15]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()`` el nombre.

Reviseu aquest concepte:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [16]:
ratings_by_title = data.groupby('title').size()
ratings_by_title

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64

Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [17]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``:

In [18]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [19]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [20]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre homes i dones:

In [21]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones:

In [22]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions:

In [23]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()

# Filter down to active_titles
rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
                                           ...   
Wrong Trousers, The (1993)               0.708666
Shawshank Redemption, The (1994)         0.700443
Great Escape, The (1963)                 0.692585
Rear Window (1954)                       0.688946
Close Shave, A (1995)                    0.667143
Name: rating, Length: 1216, dtype: float64

### Important: Temes de rendiment

Fixeu-vos en el comportament de Python i actueu en conseqüència:

In [24]:
%timeit data['title']  # Fastest
print(type(data['title']))
%timeit data.title  # Medium
print(type(data.title))
%timeit data[['title']] # Slower
print(type(data[['title']]))

701 ns ± 1.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
<class 'pandas.core.series.Series'>
1.55 µs ± 3.97 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
<class 'pandas.core.series.Series'>
7.6 ms ± 33.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
<class 'pandas.core.frame.DataFrame'>


## 3. EXERCICIS

### 3.1. EXERCICI A

+ Donada la taula ``data`` tal i com es defineix a continuació, calcula la puntuació mitjana de cada usuari i guarda-la a un ``df`` anomenat ``users_mean_rating``.

In [25]:
data_folder = 'ml-1m'

In [26]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')

data = pd.merge(pd.merge(ratings, users), movies)

users_mean_rating = data.groupby('user_id')['rating'].mean()  # Separa les puntuacións per usuaris i calcula la mitja de cada un
users_mean_rating = pd.DataFrame(data=users_mean_rating)  # Guarda les puntuacions en un nou dataFrame

+ Quina és la pel·lícula més ben puntuada (en mitja) pels usuaris? (Guarda aquest valor en una variable de tipus ``string`` anomenada ``best_movie_rating`` ).

In [27]:
movies_rating = data.groupby('title')['rating'].mean()  # Separa les puntuacions per pel·licula i calcula la mitja de cada una
movies_rating = movies_rating.sort_values(ascending=False)  # Ordena les pel·licules per puntuació descendent

best_movie_rating = movies_rating.index[0]  # Guarda la primera pel·lícula amb màxima puntuació

+ Mira si hi ha més pel·licules amb la mateixa puntuació de la més ben puntuada.

> Bloque con sangría



In [28]:
# Agafa la puntuació de la pel·lícula més votada
best_movie_rating = movies_rating.iloc[0]

# Filtra les pel·licules amb la mateixa puntuació
best_rated_movies = movies_rating[movies_rating == best_movie_rating].index

# Imprimeix per consola les pel·licules amb máxima puntuació
if len(best_rated_movies) > 1:
    print(f"Hi han {len(best_rated_movies)} amb la mateixa puntuació de {best_movie_rating}.")
    print(f"Són les següents: {best_rated_movies.values}")
else:
    print("No hi ha mes pelicules amb la mateixa puntuació")

Hi han 10 amb la mateixa puntuació de 5.0.
Són les següents: ['Ulysses (Ulisse) (1954)' 'Lured (1947)' 'Follow the Bitch (1998)'
 'Bittersweet Motel (2000)' 'Song of Freedom (1936)'
 'One Little Indian (1973)' 'Smashing Time (1967)'
 'Schlafes Bruder (Brother of Sleep) (1995)'
 'Gate of Heavenly Peace, The (1995)' 'Baby, The (1973)']


+ Busca ara aquella pel·lícula, d'entre les que tenen 5 com a puntuació mitjana, que hagi rebut més valoracions i guarda-la a una variable anomenada ``best_movie_rating_maxviews``. Així tindrem la pel·licula més ben puntuada per més usuaris.

In [29]:
# Separa les valoracions per pel·licula, fa el recompte de valoracions i de les més votades retorna la que ha sigut valorada més cops
best_movie_rating_maxviews = data.groupby('title').size()[best_rated_movies].idxmax()
print(best_movie_rating_maxviews)

Gate of Heavenly Peace, The (1995)


### 3.2. EXERCICI B

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.


In [30]:
def top_movie(dataFrame, usr):
    """
    Retorna la pel·licula millor puntuada donat un usuari

    :param dataFrame: Pandas.DataFrame amb com a mínim els id's del usuaris, els id's de les pel·licules i les puntuacions de les pel·licules
    :param usr: ID de l'usuari
    """

    user_ratings = dataFrame[dataFrame['user_id'] == usr]  # Agafa la puntuació mes alta entre totes les puntuacions fetes per l'usuari
    max_rating = user_ratings['rating'].max()  # Agafa la máxima puntuació del usuari

    return user_ratings[user_ratings['rating'] == max_rating]['title'].iloc[0]  # Retorna la primera pel·licula que troba amb la maxima puntuacuó

print(top_movie(data,1))

One Flew Over the Cuckoo's Nest (1975)


### 3.3. EXERCICI C

+ Construeix una funció que donat el dataframe ``data`` et retorni un altre dataframe ``df_counts``amb el valor que cada usuari li ha donat a una peli. Això ho farem creant un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors siguin el rating donat.

In [31]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors
    la valoració que un usuari ha donat a una peli d'un `movie_id`

    :param df: DataFrame original
    :return: DataFrame descrit adalt
    """

    # Crea un nou DataFrame amb la funcio pivot_table usant les movie_id com columne, els user_id com files i els ratings com valors
    df_counts = df.pivot_table(values='rating', index='user_id', columns='movie_id')

    return df_counts

In [32]:
df_counts = build_counts_table(data)
df_counts

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


+ Fés una funció que donada la taula anterior i dos id's (usuari i peli), extregui el valor donat:

In [33]:
def get_count(df, user_id, movie_id):
    """
    Retorna la valoració que l'usuari 'user_id' ha donat de 'movie_id'

    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param movie_id: ID de la peli
    :return: Enter amb la valoració de la peli
    """

    # Retorna la valoració del usuari user_id i la pel·licula movie_id
    return df.loc[user_id, movie_id] 

get_count(df_counts, 1, 1)

5.0

### 3.4. EXERCICI D

Si observem el nombre total d'usuaris únics i de pel.licules úniques, podem veure que els id's dels usuaris van de 1 a 6040. Normalment volem índexos que comencin al nombre 0, anant de 0 a 6039.

+ Explora els índexos de les pel·licules. **Quin problema hi ha amb els indexos de les pel·licules??**

In [34]:
unique_movies = pd.unique(data['movie_id'])
print(f'Numero de pel·licules: {len(unique_movies)}\nValor del índex més alt: {unique_movies.max()}')

Numero de pel·licules: 3706
Valor del índex més alt: 3952


> **Resposta**: El nombre de indexs és correcte, però l'index amb major valor supera per molt el número de pel·licules que hi han, per tant podem veure que els id's no estan ordenats i els valors no son coherents


+ Usant la funció `pd.Categorical(*).codes`, re-indexa els id's dels usuaris i de les pelis perquè vagin de 0 a 6039 i de 0 a 3705 respectivament:

In [35]:
# Reindexa tant els indexs dels usuaris com el de les pel·licules
data['user_id'] = pd.Categorical(data['user_id']).codes
data['movie_id'] = pd.Categorical(data['movie_id']).codes

+ Per comprovar que tot sigui correcte i guardar correctament la taula **df_counts**, torna a calcular i visualitza ``df_counts``:

In [36]:
# la vostra solució aquí
df_counts = build_counts_table(data)
df_counts

movie_id,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.5. EXERCICI E



+ Escriu una funció `distEuclid(x,y)`  que implementi la distància Euclidiana entre dos vectors usant funcions de pandas.

+ Escriu la funció `SimEuclid (DataFrame, U1, U2)` que calculin la semblança entre dos usuaris segons aquesta estructura:

    + Calcular un vector per cada usuari, $U1$ i $U2$, amb les puntuacions dels ítems comuns que han puntuat el dos usuaris.
    + Si no hi ha puntuacions en comú, retornar 0. Sinó, retornar
    
    $$\frac{1}{(1+distEuclid(U1, U2))}$$

+ Avalueu amb la funció ``%timeit`` quant triguen aquests càlculs per un parell d'usuaris.   

> *Nota: Alguns d'aquests exercicis tenen temps de càlcul de l'ordre de minuts sobre tota la base de dades. Per desenvolupar els algorismes és recomanable treballar amb una versió reduïda de la base de dades.*

Per implementar aquestes funcions únicament es permet l'ús de les funcions:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**!

In [37]:
num_movies = data.nunique()['movie_id']

def distEuclid(x, y):
    """
    Retorna la distancia euclidiana de dos vectors n-dimensionals.

    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distancia euclidiana
    """

    # Calcula la distància euclidiana utilitzant les funcions proporcionades
    return np.sqrt(np.sum((x - y)**2))


def SimEuclid(DataFrame, User1, User2):
    """
    Retorna un score que representa la similitud entre user1 i user2 basada en la distancia euclidiana

    :param DataFrame: dataframe que conté totes les dades
    :param User1: id user1
    :param User2: id user2
    :return : Escalar (float) corresponent al score
    """

    # Obté les pel·licules valoradas per cada usuari
    user1_rated_movies = DataFrame.loc[DataFrame['user_id'] == User1, 'movie_id']
    user2_rated_movies = DataFrame.loc[DataFrame['user_id'] == User2, 'movie_id']

    # Obté les pel·licules comunes entre ambdós usuaris
    common_movies = pd.merge(user1_rated_movies, user2_rated_movies)['movie_id'].values

    # Si no hi ha pel·licules en comú es retorna 0
    if len(common_movies) == 0:
      return 0

    # Obté les puntuacions de les pel·licules valoradas pels dos usuaris
    user1_values = DataFrame[(DataFrame['movie_id'].isin(common_movies)) & (DataFrame['user_id'] == User1)]['rating'].values
    user2_values = DataFrame[(DataFrame['movie_id'].isin(common_movies)) & (DataFrame['user_id'] == User2)]['rating'].values

    # Calcula la distancia euclidiana entre les dues puntuacions dels usuaris
    distance = distEuclid(user1_values, user2_values)

    # Calcula el score tenint en compte el número de pel·licules que tenen en comú vers el número total
    score = 1 / (1 + distance) * (len(common_movies)/num_movies)

    return score

In [38]:
# Execute functions
print(SimEuclid(data, 2,314))

0.0005396654074473826


In [39]:
# Execute functions
print(SimEuclid(data, 0,1))

0.0006296096420219464


In [40]:
# Execute functions
print(SimEuclid(data, 1, 5))

0.0005316828473355942


In [41]:
%timeit SimEuclid(data, 1, 5)

5.5 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 3.6. EXERCICI F

En aquest exercici desenvoluparem un sistema de recomanació col·laboratiu **basat en usuaris**.

La funció principal, ``getRecommendationsUser``, ha de tenir com a entrada una taula de puntuacions, un ``user_id``, el tipus de mesura de semblança (Euclidiana) que volem usar, el nombre `m` d'usuaris semblants que volem per fer la recomanació i el nombre ``n`` de recomanacions que volem.

Exemple: ``getRecommendationsUser(data, 2, 50, 10, SimEuclid)``

Com a sortida ha de donar la llista de les $n$ millors pel·lícules que li podriem recomanar segons la seva semblança amb altres usuaris.

> *Nota 1: S'ha d'evitar comparar ``user_id`` a ell mateix.*

> *Nota 2: Recordeu que en Python podem passar funcions com a paràmetres d'una funció.*

#### EXERCICI F.1

+ Computa la *score* de similitud del usuari desitjat (userID) respecte tots els altres i retorna un diccionari dels $m$ usuaris més propers i el seu *score*, que seran els que usarem per fer la recomanació. Normalitzeu els *scores* de sortida.

In [42]:
def find_similar_users(DataFrame, userID, m, simfunction):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.

    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param similarity: mesura de similitud
    :return : dictionary
    """

    # Exclou el propi usuari de les comparacions
    users = DataFrame[DataFrame['user_id'] != userID]['user_id'].unique()

    # Calcula la similitud entre l'usuari objectiu i tots els altres usuaris
    scores = {}
    for user in users:
        scores[user] = simfunction(DataFrame, userID, user)

    # Ordena els usuaris segons la similitud i agafa els m més semblants
    scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:m]

    return scores

In [43]:
t = datetime.datetime.now()
sim_dict = find_similar_users(data, 2, 10, SimEuclid)
t = datetime.datetime.now()-t
print(str(t))

0:00:28.826723


In [44]:
sim_dict

[(4447, 0.0016759799892629866),
 (1646, 0.0016575437514455324),
 (5830, 0.0015585655884994666),
 (2270, 0.0015274228133923392),
 (1903, 0.0015241950142672186),
 (3271, 0.001514953400110106),
 (4276, 0.001514879601095694),
 (1879, 0.0014857868522638257),
 (3625, 0.001482580170135753),
 (650, 0.0014790760952288534)]

+ Creieu que el temps de procés és assumible? Quan trigaria si ho fem per tots els usuaris?

> **RESPOSTA**: No ho és ja que aproximadament trigariem uns 163107 segons que en hores serien unes 45. Tenint en compte que 6041 usuaris no son molts, i que les grans empreses compten amb nombres d'usuaris elevats, no es una solució optima.

+ Per solucionar el problema anterior, construeix una matriu de mida $U \times U$ on cada posició $(i,j)$ indiqui la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. Calcula quant triga la teva implementació.

Heu de tenir en compte que:

* Si feu una funció que treballi amb els vectors de cada usuari i faci un doble ``for``, el procés de les dades pot trigar una bona estona.
* Si feu una funció que treballi específicament amb matrius (i no vectors) trigarà molt pocs segons. En aquest link podeu trobar indicacions de com fer-ho: https://jaykmody.com/blog/distance-matrices-with-numpy/



In [45]:
def compute_distance(fixed_arr, var_arr):
    """
    Donats dos vectors, calcula la distancia entre els subvectors formats
    pels elements en comú.
    """



In [46]:
from tqdm import tqdm

def similarity_matrix(compute_distance, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició
    indica la similitud entre usuaris (resp. ítems).

    :param df_counts: df amb els valor que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """

    df_counts = np.nan_to_num(df_counts)

    x2 = np.sum(df_counts**2, axis=1)
    xy = np.matmul(df_counts, df_counts.T)

    x2 = x2.reshape(-1, 1)
    distances = np.sqrt(x2 - 2*xy + x2.T)

    return distances

In [47]:
t = datetime.datetime.now()
sim = similarity_matrix(compute_distance, df_counts)
t = datetime.datetime.now()-t
print("Temps sense bucles:",str(t))

Temps sense bucles: 0:00:01.089490


In [93]:
sim = similarity_matrix(compute_distance, df_counts)
sim = sim / sim.sum(axis=1, keepdims=True) # Normalize the values

+ Ara torna a re-fer la funció ``find_similar_users`` i mira quant triga.

> Recorda que les scores han d'estar normalitzades!

In [94]:
def find_similar_users(DataFrame, sim_mx, userID, m):
    # la vostra solució aquí

    # Agafem la fila de la matriu corresponen a l'usuari corresponent
    user_distances = pd.Series(sim_mx[userID])

    # Ordenem els scores i agafem els més grans
    w_scores = user_distances.sort_values(ascending=False)[:m]

    return w_scores.to_dict()

In [95]:
t = datetime.datetime.now()
sim_dict = find_similar_users(data, sim, 2, 10)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.002465


In [96]:
sim_dict

{4168: 0.0005867206517490499,
 4276: 0.0005743119497839612,
 1679: 0.0005291252211625618,
 2908: 0.00045872084811123925,
 1014: 0.00045508163625882636,
 3031: 0.0004537444093801131,
 1284: 0.00044703534612834753,
 3538: 0.0004451034429672887,
 4447: 0.00044166475992387066,
 3390: 0.00044152715601305885}

#### EXERCICI F.2

+ Computa les recomanacions per un usuari concret a partir dels ratings dels seus $m$ usuaris més propers. Fes primer una funció que retorni la **weighted average list** dels $m$ usuaris més propers. Feu servir la funció anterior que usava la matriu de similituds per anar més ràpid!! (Nota: la **weighted average list** es calcularà agregant els $n$ items més puntuats de cadascun dels m users més semblants al usuari donat).

In [97]:
from collections import defaultdict

def weighted_average(DataFrame, user, sim_mx, m):
    """
    df_counts: usuaris_semblants * pelis
    similarity: {user: similarity}

    Aquesta funcio calculara el score predit (Score que ha donat l'usuariB * similitud(A,B)) i despres sumara
    tots els valors per cada peli; retornara {peli_id: suma de score predit}
    """

    # Creem un diccionari per guardar les pel·licules recomenades i les seves scores
    movies_score = defaultdict(int)

    # Agafem els m usuaris més semblants al usuari 'user'
    similar_users = find_similar_users(DataFrame, sim_mx, user, m)

    # Obtenim les pel·licules valorades per l'usuari 'user'
    userA_rated_movies = DataFrame.loc[DataFrame['user_id'] == user, 'movie_id'].values

    # Calculem la perdicció dels scores de pel·licules desconegudes per l'usuari 'user' en base als 'm' usuaris més semblats a ell
    for userB, similarity in similar_users.items():

      # Agafem les  pel·liculas valorades per l'usuari B
      userB_rated_movies = DataFrame.loc[DataFrame['user_id'] == userB, 'movie_id'].values

      # Eliminem les pel·licules valorades per tots dos
      movies = np.setdiff1d(userB_rated_movies, userA_rated_movies, assume_unique=True)

      

      # Obtenim les puntuaciones 
      userB_values = DataFrame[(DataFrame['movie_id'].isin(movies)) & (DataFrame['user_id'] == userB)][['movie_id', 'rating']].values
      
      # Actualitzem l'score de la pel·licula recomenada
      movies_score.update({movie: movies_score[movie] + rating * similarity for movie, rating in userB_values})

    return movies_score

In [98]:
def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """
    Retorna un dataframe de pel·licules amb els scores.

    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_function
    :param n: nombre de pelis a recomanar
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return : pandas de pelis amb els seus scores predits
    """

    # Agafem les prediccions de les pel·licules no vistes per l'usuari 'user'
    predictions = list(weighted_average(DataFrame, user, sim_mx, m).items())

    # Ordenem les recomenacions en funció del score
    predictions.sort(reverse=True, key=lambda x: x[1])

    # Agafem les 'n' millors prediccions 
    predictions = predictions[:n]

    # Creem un diccionari on guardar les recomenacions
    recomendations = {'movie_id': [], 'title': [], 'score': []}
    
    # Afegim titol a les recomenacions
    for m_id, score in predictions:
        m_title = np.unique(DataFrame[DataFrame['movie_id'] == m_id]['title'])[0]

        # Actualirzem les recomenacions
        recomendations['movie_id'].append(m_id)
        recomendations['title'].append(m_title)
        recomendations['score'].append(score)
    
    return pd.DataFrame(recomendations)

In [99]:
t = datetime.datetime.now()
user_prediction = getRecommendationsUser(data, 3, sim, 10, 50)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.246876


In [100]:
user_prediction

,movie_id,title,score
0,579,"Silence of the Lambs, The (1991)",0.105204
1,1485,L.A. Confidential (1997),0.103395
2,802,"Godfather, The (1972)",0.103323
3,287,Pulp Fiction (1994),0.102716
4,593,Fargo (1996),0.100621
5,858,"Wizard of Oz, The (1939)",0.099163
6,2374,"Matrix, The (1999)",0.098296
7,1178,Back to the Future (1985),0.096294
8,2557,"Sixth Sense, The (1999)",0.095588
9,575,Terminator 2: Judgment Day (1991),0.095445


### 3.7. EXERCICI G


A continuació usarem la metrica **Mean Absolute Error (MAE)** per evaluar el nostre sistema. Aquesta mètrica ens permetrà mesurar la diferencia entre dues llistes donat un usuari:
+ La llista amb els ratings originals d'un usuari donat
+ La llista de les prediccions generades per aquest usuari

#### EXERCICI G.1

+ Treu el 10% dels usuaris i reserva aquests en una variable anomenada ``test_set`` i la resta en una variable anomenada ``train_set``.

In [56]:
import random
unique_users = data['user_id'].unique()
n_test_users = int(len(unique_users)*0.1)
n_train_users = len(unique_users) - n_test_users

test_users = random.sample(list(unique_users), n_test_users)

test_set = data[data['user_id'].isin(test_users)]
train_set = data[~data['user_id'].isin(test_users)]

assert test_set.shape[0] + train_set.shape[0] == data.shape[0]

+ Què passarà si calculo la matriu de similitud amb ``train_set`` i després intento predir pels usuaris de ``test_set``??

> **Resposta**:


#### EXERCICI G.2

+ Selecciona aproximadament el 80% de les interaccions de cada usuari de test i junta-les al ``train_set``. Podem ara evaluar el sistema?


> Com la pràctica és molt llarga us donem el codi per un usuari donat i vosaltres només heu de crear la funció que, per cada usuari, afagi el 80% de les intraccions i les unifiqui al dataframe de train.

In [57]:
test_set.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
15,48,1104,4,978813972,M,18,12,77084,One Flew Over the Cuckoo's Nest (1975),Drama
16,52,1104,5,977946400,M,25,0,96931,One Flew Over the Cuckoo's Nest (1975),Drama
19,58,1104,4,977934292,F,50,1,55413,One Flew Over the Cuckoo's Nest (1975),Drama
21,79,1104,4,977786172,M,56,1,49327,One Flew Over the Cuckoo's Nest (1975),Drama
41,165,1104,5,977299851,M,18,4,92802,One Flew Over the Cuckoo's Nest (1975),Drama


In [58]:
# Agafem el 20% de les pelis que ha consumit cada usuari de test
groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2
groupby_count

user_id
4       39.6
10      27.4
33      32.8
45       8.2
48      21.6
        ... 
5980    56.0
5986    11.8
5994    37.2
5998     5.4
6000    46.8
Name: movie_id, Length: 604, dtype: float64

Seleccionem la posició 1 i aquest user_id serà el que usarem pel codi d'exemple (que després haureu de replicar).

In [59]:
groupby_count.reset_index().iloc[0]

user_id      4.0
movie_id    39.6
Name: 0, dtype: float64

In [60]:
n_test_samples = int(groupby_count.reset_index().iloc[1]['movie_id'])
u = groupby_count.reset_index().iloc[1]['user_id']

In [61]:
test_set_user = test_set[test_set['user_id'] == u]
frame_test = test_set_user.sample(n_test_samples)
print("TOTAL SAMPLES OF THE USER: " + str(len(test_set_user)))
print("TOTAL SAMPLES OF THE USER IN TEST SET: " + str(len(frame_test)))

TOTAL SAMPLES OF THE USER: 137
TOTAL SAMPLES OF THE USER IN TEST SET: 27


In [137]:
len(test_set_user.index)

137

In [63]:
frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
print("TOTAL SAMPLES OF THE USER IN TRAIN SET: " + str(len(frame_train)))

TOTAL SAMPLES OF THE USER IN TRAIN SET: 110


In [64]:
assert len(frame_train) + len(frame_test) == len(test_set_user)

In [171]:
def add_testdata(traindf, test_set):
    """
    Retorna els N usuaris més similars basat en la correlació de Pearson (no)

    :param traindf: dataframe que conté les dades de train
    :param test_set: dataframe que conté les dades de test

    :return :
        - :param 1st: dataframe que conté les dades de train juntament amb el 80% de test seleccionat
        - :param 2nd: dataframe que conté les dades de test que queden (20% restant)
    """

    # Agafem el 20% de les pelis que ha valorat cada usuari
    groupby_count = test_set.groupby('user_id')['movie_id'].count() * 0.2
    users_views = groupby_count.reset_index()

    rest_dict = {key: [] for key in traindf.columns}
    train_dict = {key: [] for key in traindf.columns}

    for user, views in tqdm(users_views.itertuples(index=False), total=len(users_views)):

        
        test_set_user = test_set[test_set['user_id'] == user]
        frame_test = test_set_user.sample(int(views))
        frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]

        for key in traindf.columns.values:
            rest_dict[key].extend(frame_test[key])
            train_dict[key].extend(frame_train[key])

    rest = pd.DataFrame(rest_dict)
    traindf = pd.concat([traindf, pd.DataFrame(train_dict)], ignore_index=True)

    return traindf, rest


In [101]:
train, test = add_testdata(train_set, test_set)

100%|██████████| 604/604 [00:00<00:00, 2407.35it/s]


In [141]:
test['user_id'].isin(train['user_id']).all()

True

In [67]:
train

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,0,1104,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,1104,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,11,1104,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,14,1104,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,16,1104,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
979700,6000,2295,4,956810425,F,25,7,94117,Peeping Tom (1960),Drama|Horror|Thriller
979701,6000,3313,4,956807744,F,25,7,94117,Prick Up Your Ears (1987),Drama
979702,6000,3700,4,1004811080,F,25,7,94117,Get Carter (1971),Thriller
979703,6000,640,3,956810840,F,25,7,94117,Fear (1996),Thriller


In [68]:
test

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,4,258,3,978245037,M,25,20,55455,Like Water for Chocolate (Como agua para choco...,Drama|Romance
1,4,1413,4,978245513,M,25,20,55455,Love! Valour! Compassion! (1997),Drama|Romance
2,4,1356,3,978245695,M,25,20,55455,Donnie Brasco (1997),Crime|Drama
3,4,2144,5,978243121,M,25,20,55455,Velvet Goldmine (1998),Drama
4,4,2078,2,978246033,M,25,20,55455,"Few Good Men, A (1992)",Crime|Drama
...,...,...,...,...,...,...,...,...,...,...
20499,6000,2401,4,1004811365,F,25,7,94117,eXistenZ (1999),Action|Sci-Fi|Thriller
20500,6000,2651,5,956801861,F,25,7,94117,American Beauty (1999),Comedy|Drama
20501,6000,2999,3,956811341,F,25,7,94117,Pacific Heights (1990),Thriller
20502,6000,1126,3,956802336,F,25,7,94117,"Big Blue, The (Le Grand Bleu) (1988)",Adventure|Romance


In [69]:
train.shape

(979705, 10)

In [70]:
test.shape

(20504, 10)

In [71]:
data.shape

(1000209, 10)

In [72]:
train.shape[0] + test.shape[0]

1000209

In [73]:
assert train.shape[0] + test.shape[0] == data.shape[0]

#### EXERCICI G.3

+ Fes una funció que serveixi per evaluar el nostre sistema usant la mètrica MAE.

In [168]:
def evaluateRecommendations(train, test, m,n, sim):
    """
    Retorna l'error generat pel model

    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a retornar (no)
    :param sim: matriu de similitud
    :return : Escalar (float) corresponent al MAE
    """

    # Agafem un usuari del conjunt de test
    user = test['user_id'][0]

    # Agafem les recomanacions generadas a partir de la matriz de similitudes y el conjunt de dades de entrenament
    recomendations = getRecommendationsUser(train, user, sim, n, m)
    movies = recomendations['movie_id']

    # Cogemos del conjunto de prueba las puntuaciones de las valoraciones del usuario de las peliculas que han sido recomendadas
    test_ratings = test[(test['user_id'] == user) & (test['movie_id'].isin(movies))]['rating']

    # Cogemos las puntuaciones generadas por la recomendación
    train_ratings = recomendations['score']

    return (train_ratings - test_ratings).sum() / n

    
    



In [169]:
t = datetime.datetime.now()
mae = evaluateRecommendations(train, test, 50, 10, sim)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.270517


In [170]:
mae

0.0

### 3.8. EXERCICI H (exercici opcional, no obligatori)


+ **Que surt més a compte, fer un recomanador unic pels dos sexes o un per cada sexe?** Justifica la resposta per escrit i amb el codi necessari.